In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Load the dataset
data = pd.read_csv("/home/mw/.cache/kagglehub/datasets/yasserh/housing-prices-dataset/versions/1/Housing.csv")
data = data.dropna()

# Handle categorical columns
categorical_columns = data.select_dtypes(include=['object']).columns
print(categorical_columns)

for col in categorical_columns:
    if data[col].nunique() <= 2:
        data[col] = LabelEncoder().fit_transform(data[col])
    else:
        one_hot = pd.get_dummies(data[col], prefix=col)
        data = pd.concat([data.drop(col, axis=1), one_hot], axis=1)

# Handle boolean columns
boolean_columns = data.select_dtypes(include='bool').columns
print(boolean_columns)
data[boolean_columns] = data[boolean_columns].astype(int)
print(data.dtypes)

# Standardize numeric columns
numeric_columns = data.select_dtypes(include=['float64']).columns
data[numeric_columns] = (data[numeric_columns] - data[numeric_columns].mean()) / data[numeric_columns].std()

# Standardize specific columns
data['area_standardized'] = (data['area'] - data['area'].mean()) / data['area'].std()
data['bedrooms_standardized'] = (data['bedrooms'] - data['bedrooms'].mean()) / data['bedrooms'].std()
data['bathrooms_standardized'] = (data['bathrooms'] - data['bathrooms'].mean()) / data['bathrooms'].std()
data['stories_standardized'] = (data['stories'] - data['stories'].mean()) / data['stories'].std()
data['parking_standardized'] = (data['parking'] - data['parking'].mean()) / data['parking'].std()

# Drop the original columns
data = data.drop(columns=['area', 'bedrooms', 'bathrooms', 'stories', 'parking'])

print(data.columns)

# Define target and features
target_column = "price"
features = data.drop(target_column, axis=1)
target = data[target_column]

print(features.head())

# Convert features and target to TensorFlow tensors
features_tensor = tf.convert_to_tensor(features.values, dtype=tf.float32)
target_tensor = tf.convert_to_tensor(target.values, dtype=tf.float32)

# Create a TensorFlow Dataset
dataset = tf.data.Dataset.from_tensor_slices((features_tensor, target_tensor))
dataset = dataset.shuffle(buffer_size=len(data)).batch(128)


2025-01-05 05:54:49.573991: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-05 05:54:49.735536: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736036689.865034    7518 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736036689.892849    7518 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-05 05:54:50.093975: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Index(['mainroad', 'guestroom', 'basement', 'hotwaterheating',
       'airconditioning', 'prefarea', 'furnishingstatus'],
      dtype='object')
Index(['furnishingstatus_furnished', 'furnishingstatus_semi-furnished',
       'furnishingstatus_unfurnished'],
      dtype='object')
price                              int64
area                               int64
bedrooms                           int64
bathrooms                          int64
stories                            int64
mainroad                           int64
guestroom                          int64
basement                           int64
hotwaterheating                    int64
airconditioning                    int64
parking                            int64
prefarea                           int64
furnishingstatus_furnished         int64
furnishingstatus_semi-furnished    int64
furnishingstatus_unfurnished       int64
dtype: object
Index(['price', 'mainroad', 'guestroom', 'basement', 'hotwaterheating',
       'airconditioni

I0000 00:00:1736036694.237257    7518 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2171 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [2]:
import tensorflow as tf
from keras.src import layers, Model, optimizers
import tensorflow as tf

class Discriminator(tf.keras.Model):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.net = tf.keras.Sequential([
            layers.Dense(256, input_dim=input_dim),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2),
            layers.Dense(128),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2),
            layers.Dense(1)  # Output layer for binary classification (real or fake)
        ])
        
    def call(self, x):
        features = x  # Start with the input as the initial features
        for i, layer in enumerate(self.net.layers[:-1]):
            features = layer(features)  # Apply each layer to the features
        
        logits = self.net.layers[-1](features)  # Apply the final Dense layer to get logits
        output = tf.sigmoid(logits)  # Apply sigmoid to get the binary output (real or fake)
        return output, features  # Return the output and the features (intermediate layer outputs)


class Encoder(Model):
    def __init__(self, input_dim, latent_dim):
        super(Encoder, self).__init__()
        
        self.fc1 = layers.Dense(512, input_dim=input_dim, activation='relu')
        self.fc2 = layers.Dense(256, activation='relu')
        self.fc3 = layers.Dense(128, activation='relu')
        self.fc_mean = layers.Dense(latent_dim)
        self.fc_log_var = layers.Dense(latent_dim)

    def call(self, x):
        print(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        mean = self.fc_mean(x)
        log_var = self.fc_log_var(x)
        return mean, log_var

class Decoder(Model):
    def __init__(self, latent_dim, output_dim):
        super(Decoder, self).__init__()
        
        self.fc1 = layers.Dense(128, input_dim=latent_dim, activation='relu')
        self.fc2 = layers.Dense(256, activation='relu')
        self.fc3 = layers.Dense(512, activation='relu')
        self.fc_output = layers.Dense(output_dim, activation='sigmoid')

    def call(self, z):
        z = self.fc1(z)
        z = self.fc2(z)
        z = self.fc3(z)
        reconstructed_x = self.fc_output(z)
        return reconstructed_x

class VAE_GAN(Model):
    def __init__(self, encoder, decoder, discriminator):
        super(VAE_GAN, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.discriminator = discriminator
        
    def reparameterize(self, mu, logvar):
        std = tf.exp(0.5 * logvar)
        eps = tf.random.normal(shape=tf.shape(std))
        return mu + eps * std
        
    def call(self, x):
        # Encode
        print(x)
        z_mean, z_log_var = self.encoder(x)
        
        # Reparameterize
        z = self.reparameterize(z_mean, z_log_var)
        # Decode
        x_reconstructed = self.decoder(z)
        
        # Discriminate both real and reconstructed
        # real_or_fake_input = tf.concat([x, x_reconstructed], axis=-1)
        real_or_fake_reconstructed, features_constructed = self.discriminator(x_reconstructed)
        real_or_fake_real, features_real = self.discriminator(x)
        
        return (
            x_reconstructed,
            real_or_fake_reconstructed,
            real_or_fake_real,
            z_mean,
            z_log_var
        )


In [3]:
from tqdm import tqdm

In [4]:
def train_vae_gan(model, dataset, epochs=200, learning_rate=1e-3):
    # Define optimizers
    optimizer_enc = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    optimizer_dec = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    optimizer_dis = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    print('entered')

    for epoch in range(epochs):
        print("Trainable variables in the model:")
        for var in model.trainable_variables:
            print(var.name)  
            
        # print(model.summary())# Prints the name of each trainable variable

        running_vae_loss = 0
        running_dis_loss = 0
        running_gen_loss = 0
        
        for batch in tqdm(dataset, desc=f'Epoch {epoch + 1}/{epochs}'):
            inputs = batch[0]  # Extract inputs (e.g., images)

            # Train Discriminator
            with tf.GradientTape() as tape_dis:
                x_reconstructed, real_or_fake_reconstructed, real_or_fake_real, z_mean, z_log_var = model(inputs)
                
                # Labels for discriminator
                real_labels = tf.ones_like(real_or_fake_real)
                fake_labels = tf.zeros_like(real_or_fake_reconstructed)
                
                # Compute Discriminator Loss
                dis_real_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                    logits=real_or_fake_real, labels=real_labels))
                dis_fake_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                    logits=real_or_fake_reconstructed, labels=fake_labels))
                dis_loss = dis_real_loss + dis_fake_loss
            
            gradients_dis = tape_dis.gradient(dis_loss, model.discriminator.trainable_variables)
            optimizer_dis.apply_gradients(zip(gradients_dis, model.discriminator.trainable_variables))
            
            # Train Generator (Decoder)
            with tf.GradientTape() as tape_dec:
                gen_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                    logits=real_or_fake_reconstructed, labels=real_labels))
            
            gradients_dec = tape_dec.gradient(gen_loss, model.decoder.trainable_variables)
            optimizer_dec.apply_gradients(zip(gradients_dec, model.decoder.trainable_variables))

            # Train VAE (Encoder + Decoder)
            with tf.GradientTape() as tape_vae:
                # Reconstruction Loss
                recon_loss = tf.reduce_mean(tf.square(x_reconstructed - inputs))
                # KL-Divergence Loss
                kl_loss = -0.5 * tf.reduce_mean(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
                vae_loss = recon_loss + kl_loss
            
            gradients_enc = tape_vae.gradient(vae_loss, model.encoder.trainable_variables)
            optimizer_enc.apply_gradients(zip(gradients_enc, model.encoder.trainable_variables))

            # Track losses
            running_dis_loss += dis_loss.numpy()
            running_gen_loss += gen_loss.numpy()
            running_vae_loss += vae_loss.numpy()

        # Print average losses per epoch
        print(f'Epoch [{epoch + 1}/{epochs}]: D: {running_dis_loss / len(dataset):.4f}, G: {running_gen_loss / len(dataset):.4f}, VAE: {running_vae_loss / len(dataset):.4f}')

        # Save model weights every 10 epochs
        if (epoch + 1) % 10 == 0:
            model.save_weights(f'vae_gan_epoch_{epoch + 1}.h5')


In [5]:


# Assuming features_tensor is defined and dataset is provided
input_dim = features_tensor.shape[1]


encoder = Encoder(input_dim, latent_dim=10)
decoder = Decoder(latent_dim=10, output_dim=input_dim)
discriminator = Discriminator(input_dim)
input_example = tf.random.normal([128, input_dim])  # Replace with your actual batch size and input dimension


model = VAE_GAN(encoder, decoder, discriminator)
mu,logvar = model.encoder(input_example)

std = tf.exp(0.5 * logvar)
eps = tf.random.normal(shape=tf.shape(std))
encoder_output = mu + eps * std
decoder_output = model.decoder(encoder_output)
print(type(decoder_output))  # This should print <class 'tensorflow.python.framework.ops.EagerTensor'>

discriminator_output = model.discriminator(decoder_output)
print("Encoder output shape:", logvar.shape)
print("Decoder output shape:", decoder_output.shape)
print("Discriminator output shape:", discriminator_output[0].shape)
print(model.summary())
print(model.decoder.trainable)
model.decoder.trainable = True
model.encoder.trainable = True
model.discriminator.trainable = True


device = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'

train_vae_gan(model, dataset, epochs=200, learning_rate=1e-3)


/home/mw/miniconda3/envs/research/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


tf.Tensor(
[[-0.8728698   0.7514581  -1.9147625  ...  0.25894696 -1.1722629
  -1.2254761 ]
 [-0.26744348 -0.15550892  0.01041336 ...  0.09169319  0.36868748
   0.74312085]
 [-0.08855354  1.0123518  -0.93546706 ... -0.26214638 -1.4083754
  -0.2827134 ]
 ...
 [ 0.6975861   0.19835013  0.7918325  ... -1.3251573  -0.5483585
   0.8277462 ]
 [ 0.8718931   0.00763387 -0.6905358  ... -0.1721974   0.88182336
   1.8044299 ]
 [-0.33327076  0.3534472   0.43091783 ...  0.05074676 -0.4394912
  -1.2760217 ]], shape=(128, 14), dtype=float32)
tf.Tensor(
[[-0.8728698   0.7514581  -1.9147625  ...  0.25894696 -1.1722629
  -1.2254761 ]
 [-0.26744348 -0.15550892  0.01041336 ...  0.09169319  0.36868748
   0.74312085]
 [-0.08855354  1.0123518  -0.93546706 ... -0.26214638 -1.4083754
  -0.2827134 ]
 ...
 [ 0.6975861   0.19835013  0.7918325  ... -1.3251573  -0.5483585
   0.8277462 ]
 [ 0.8718931   0.00763387 -0.6905358  ... -0.1721974   0.88182336
   1.8044299 ]
 [-0.33327076  0.3534472   0.43091783 ...  0.05074

Model: "vae_gan"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder (Encoder)               │ ?                      │       174,484 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Decoder)               │ ?                      │       173,198 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ discriminator (Discriminator)   │ ?                      │        38,401 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 386,083 (1.47 MB)

...

None
True
entered
Trainable variables in the model:
kernel
bias
kernel
bias
kernel
bias
kernel
bias
kernel
bias
kernel
bias
kernel
bias
kernel
bias
kernel
bias
kernel
bias
gamma
beta
kernel
bias
gamma
beta
kernel
bias


Epoch 1/200:   0%|          | 0/5 [00:00<?, ?it/s]

tf.Tensor(
[[ 1.          0.          1.         ... -0.56966335 -0.9285436
  -0.8050017 ]
 [ 1.          0.          0.         ... -0.56966335  2.5296998
  -0.8050017 ]
 [ 1.          0.          0.         ... -0.56966335  0.22420415
   0.3556489 ]
 ...
 [ 1.          1.          0.         ... -0.56966335  1.3769519
  -0.8050017 ]
 [ 1.          0.          1.         ...  1.4205067  -0.9285436
  -0.8050017 ]
 [ 1.          0.          0.         ... -0.56966335 -0.9285436
  -0.8050017 ]], shape=(128, 14), dtype=float32)
tf.Tensor(
[[ 1.          0.          1.         ... -0.56966335 -0.9285436
  -0.8050017 ]
 [ 1.          0.          0.         ... -0.56966335  2.5296998
  -0.8050017 ]
 [ 1.          0.          0.         ... -0.56966335  0.22420415
   0.3556489 ]
 ...
 [ 1.          1.          0.         ... -0.56966335  1.3769519
  -0.8050017 ]
 [ 1.          0.          1.         ...  1.4205067  -0.9285436
  -0.8050017 ]
 [ 1.          0.          0.         ... -0.5696633

Epoch 1/200:   0%|          | 0/5 [00:00<?, ?it/s]


ValueError: No gradients provided for any variable.